# Rust で実装した DQRTAのシミュレーションを JupyterLab 上で可視化して共有する

ドラゴンクエストシリーズのRTAにおいて、解析結果で得られた情報や、それを使ってシミュレーションした結果を可視化して共有したい場合がよくあります。<br>
昔は結果をcsvとかで出力して、それをExcelシートやスプレッドシートにimportしてグラフ作成とかしていましたが、どうせならもっとスマートかつオシャレにやりたいですね。

ということで、低レイヤーかつ高速なシミュレーションと、今どきのデータサイエンスを組み合わせるべく、下記のようにやってみました。

* コア機能は `Rust` で実装
* [evcxr-jupyter](https://github.com/google/evcxr/tree/master/evcxr_jupyter) を使って `JupyterLab` 上で実行
* グラフは `plotter` で描画

## 1. コア機能を Rust で実装

ROM解析の結果をシミュレーションするようなライブラリを実装します。さて、早速プログラミング言語やフレームワークを使うか決める必要がありますが、今回の場合は下記の要件が重要になってきそうです。

1. ビット演算やビット精度など、低レイヤーな振る舞いまで再現したい。
2. 高速にシミュレーションを回したい。
3. 実装したライブラリを使って、PWAなシミュレータやツールも公開出来るようにしたい。

僕の中では1, 2の時点でほぼ C/C++ か Rust に絞られました(あとはあるとしたらGolang?)。さて、残る3番目の要件が決め手になります。つまり、実装したコアライブラリをJavaScriptから叩きたいのですが、`WebAssembly` を使うとスマートに実現出来そうですね。そして、WASMといえばRustが一番対応が進んでいそう、ということでRustを使うことに決めました。C++も `Emscripten` を使えばWASMも出力できますが、ABIのせいで毎回 `extern C` しなきゃいけないのが面倒だよね...

ということで、まずはSFC版DQ3の乱数生成の仕様を解析して、[シミュレータを実装しました](https://github.com/Maru0137/dq3sim)。 [wasm-binding](https://github.com/rustwasm/wasm-bindgen) で関数や構造体を簡単にbindingして、 [wasm-pack](https://github.com/rustwasm/wasm-pack)で簡単にnpmパッケージをビルドしてpublishできて、とてもお手軽でした。

# 2. Jupyter Lab 上で実行

プレイヤーに実際に使ってもらうようなツールはフロントエンドを実装して公開すればよいのですが、文章も混ざった調査結果みたいなものは [Jupyter](https://jupyter.org/)みたいなものを使うほうが良さそうです。調べてみると、Rustのインタプリタである `evcxr` を使ったKernelの `evcxr-jupyter` を使って、Jupyter上でRustが使えそうな事がわかりました。

ということで、早速インストールして試してみましょう。

### 2.1. Crateのimport
1 で実装したコアライブラリのCrateをimportしましょう。 `:dep` コマンドで `Cargo` で定義しているようなdependencyが記述できて、crateがimportできるようです。

In [2]:
:dep dq3 = { path = "../" }
use dq3::rand::Rng;

さっそく乱数生成器を生成して、乱数値を表示してみましょう。

In [3]:
let mut rng = Rng::new(None);
println!("{:x}", rng.state());

aae21259


ちゃんと初期値が表示されました、やったね！<br>
`:var` コマンドで変数一覧も表示できるようです。

In [4]:
:vars

Variable,Type
rng,dq3::rand::Rng


### 2.2. 実装した乱数生成器を使って、エンカウント歩数初期値の分布をシミュレーション

身近な例として、SFC版DQ3のエンカウント歩数初期値の分布をシミュレーションしてみましょう。今までも[仕様の解析や分布が計算はされています](https://ch.nicovideo.jp/mugensai/blomaga/ar161397)が、ここでのモチベーションは、実際の乱数生成器のもとでシミュレーションするとどうなるのかを確認することです。なぜこんなことをやるかは、例えば [この文章](http://www001.upp.so-net.ne.jp/isaku/rand.html)などを参照してください。 

ということで、まずは`[0, 16]` の区間の乱数列を100万サンプル生成してみましょう。このNotebookのコンセプトはあくまでJupyterを試すことなので、乱数生成の仕様は別の機会で説明することとします。

(本当はvecを表示したかったのですが、Python版のような要素の省略と展開が出来ず、100万要素の乱数全てが表示されてしまったので、ここでは省略します。)

In [5]:
let upper_by_trial: u8 = 16;
let trial: usize = 3;
let upper_sum: u8 = upper_by_trial * (trial as u8);
let sample: usize = 1000000;
let sequence_size = sample + trial - 1;

let mut rng = Rng::new(None);
let mut sequence = Vec::new();
for _ in 0..sequence_size {
    sequence.push(rng.rand_by_multiply(upper_by_trial));
}

()

そこから、連続する3つの乱数列を取り出して加算すると、エンカウント歩数の初期値の乱数になります。

In [6]:
let mut encount_steps = Vec::new();
for i in 0..sample {
    let slice = &sequence[i..i+trial];
    let encount_step = slice.iter().fold(0, |acc, v| acc + *v);
    encount_steps.push(encount_step);
}

()

### 2.3. エンカウント歩数の分布を描画する

分布を描画してみましょう。 Rustのグラフ描画のcrateを探したところ、 [plotter](https://docs.rs/plotters/0.2.12/plotters/index.html)が一番良さそうだったのでこれを使ってみます。

In [7]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "histogram"] }
use plotters;
use plotters::prelude::*;

let figure = evcxr_figure((960, 540), |root| {
    root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Distribution of Encount Step", ("sans-serif", 30.0).into_font())
        .build_ranged(0u32..upper_sum as u32, 0f32..5f32)?;

    chart
        .configure_mesh()
        .y_label_formatter(&|y| format!("{}%", *y))
        .disable_x_mesh()
        .disable_y_mesh()
        .x_label_offset(30)
        .y_desc("Probability")
        .x_desc("Encount Step")
        .axis_desc_style(("sans-serif", 15).into_font())
        .draw()?;
    
    chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
            .data(encount_steps.iter().map(|x: &u8| (*x as u32, 100f32 / sample as f32))),
    )?;
    
    Ok(())
});
figure

Distribution of Encount Step
 
 
Probability
 
 
Encount Step
 
 
 
0.5%
 
 
 
1%
 
 
 
1.5%
 
 
 
2%
 
 
 
2.5%
 
 
 
3%
 
 
 
3.5%
 
 
 
4%
 
 
 
4.5%
 
 
 
 
0
 
 
 
10
 
 
 
20
 
 
 
30
 
 
 
40

ちゃんと描画できました！<br>
実際の乱数生成器を使った場合でも、そこまで偏った分布にはならなさそうというのが分かります。

## 3. まとめ

割と上手くいったので、今後はこんな感じで調査結果の記事とかを書くのがよさそうです。
文章や数式も Markdown 使って簡単に書けるので、とても良さそうです。

一方で、下記の方な不満点も少しはありました。

* crateの読み込みをすると、割と実行時間がかかる
* VSCodeのJupyter ExtensionがRustに対応していない
  * 補完とか出来なくて辛い...
* Rustにグラフ描画
  * `matplotlib` のような手軽さか、 `d3.js` のような高機能さに比べると、ちょっと物足りない...
  * IPythonのように、 JavaScriptのコードをJupyter上でで実行出来たら嬉しいな。
 
Rustは勢いもあるので、今後の発展に期待することとします。